In this lab you will do the following steps in order:

1. Explore and Load dataset.
2. Train [faster RCNN](https://towardsdatascience.com/faster-r-cnn-for-object-detection-a-technical-summary-474c5b857b46) for uno cards object detection.
3. Test the network.




Use GPU if available

In [ ]:
import torch
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')
DEVICE = torch.device("cuda:0" if train_on_gpu else "cpu")
print(DEVICE)

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import re

1. Explore and Load dataset.

In [ ]:
!gdown "13IjqxWIVsP2bgPZP6sE9_VYGXMeu1MWI"
!unzip -q -n "/content/Uno Cards.v2-raw.voc.zip" -d data

Setup of the training procedure

In [ ]:
import torch
BATCH_SIZE = 8 # increase / decrease according to GPU memeory
RESIZE_TO = 416 # resize the image for training and transforms
NUM_EPOCHS = 1 # number of epochs to train for
NUM_WORKERS = 4
# training images and XML files directory
TRAIN_DIR = 'data/train'
# validation images and XML files directory
VALID_DIR = 'data/valid'
TEST_DIR = 'data/test'
# classes: 0 index is reserved for background
CLASSES = ['__background__','0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14']

NUM_CLASSES = len(CLASSES)
# whether to visualize images after crearing the data loaders
VISUALIZE_TRANSFORMED_IMAGES = True
# location to save model and plots
OUT_DIR = 'outputs'

Utils

In [ ]:
from IPython.lib.display import exists
import albumentations as A
import cv2
import numpy as np
import torch
import os
import matplotlib.pyplot as plt
from albumentations.pytorch import ToTensorV2
plt.style.use('ggplot')
# this class keeps track of the training and validation loss values...
# ... and helps to get the average for each epoch as well
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0
class SaveBestModel:
    """
    Class to save the best model while training. If the current epoch's
    validation loss is less than the previous least less, then save the
    model state.
    """
    def __init__(
        self, best_valid_loss=float('inf')
    ):
        self.best_valid_loss = best_valid_loss

    def __call__(
        self, current_valid_loss,
        epoch, model, optimizer
    ):
        if current_valid_loss < self.best_valid_loss:
            self.best_valid_loss = current_valid_loss
            print(f"\nBest validation loss: {self.best_valid_loss}")
            print(f"\nSaving best model for epoch: {epoch+1}\n")
            os.makedirs("outputs",exist_ok=True)
            torch.save({
                'epoch': epoch+1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                }, 'outputs/best_model.pth')
def collate_fn(batch):
    """
    To handle the data loading as different images may have different number
    of objects and to handle varying size tensors as well.
    """
    return tuple(zip(*batch))
# define the training tranforms
def get_train_transform():
    return A.Compose([
        A.Flip(0.5),
        A.RandomRotate90(0.5),
        A.MotionBlur(p=0.2),
        A.MedianBlur(blur_limit=3, p=0.1),
        A.Blur(blur_limit=3, p=0.1),
        ToTensorV2(p=1.0),
    ], bbox_params={
        'format': 'pascal_voc',
        'label_fields': ['labels']
    })
# define the validation transforms
def get_valid_transform():
    return A.Compose([
        ToTensorV2(p=1.0),
    ], bbox_params={
        'format': 'pascal_voc',
        'label_fields': ['labels']
    })
def show_transformed_image(train_loader):
    """
    This function visualizes transformed images from the `train_loader`.

    It helps to verify if the transformations applied to the images and their
    corresponding labels are correct. The function only executes if the flag
    `VISUALIZE_TRANSFORMED_IMAGES` is set to `True` in the configuration file (`config.py`).

    Args:
        train_loader (torch.utils.data.DataLoader): The data loader containing
                                                  transformed training images and labels.
    """

    if len(train_loader) > 0:
        # Iterate over the data loader to get one batch of images and labels
        for i in range(1):
            images, targets = next(iter(train_loader))

            # Move images to the specified device (CPU or GPU)
            images = list(image.to(DEVICE) for image in images)

            # Move labels to the specified device and convert non-string elements to tensors on that device
            targets = [{k: v.to(DEVICE) for k, v in t.items() if not isinstance(v, str)} for t in targets]

            # Extract bounding boxes and labels from the first target in the batch
            boxes = targets[i]['boxes'].cpu().numpy().astype(np.int32)
            labels = targets[i]['labels'].cpu().numpy().astype(np.int32)

            # Get the first image from the batch and convert it to a NumPy array for visualization
            sample = images[i].permute(1, 2, 0).cpu().numpy()

            # Draw bounding boxes and labels on the image
            for box_num, box in enumerate(boxes):
                cv2.rectangle(sample,
                             (box[0], box[1]),  # Top-left corner coordinates
                             (box[2], box[3]),  # Bottom-right corner coordinates
                             (0, 0, 255), 2)    # Red color, thickness 2
                label_text = CLASSES[labels[box_num]]  # Get label text from label index
                cv2.putText(sample, label_text,
                            (box[0], box[1] - 10),  # Position above top-left corner
                            cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 2)  # Font style, size, color, thickness

            # Display the image using matplotlib
            plt.imshow(sample)
            plt.axis("off")  # Hide axes for cleaner visualization
            plt.show()

def save_model(epoch, model, optimizer):
    """
    Function to save the trained model till current epoch, or whenver called
    """
    os.makedirs("outputs",exist_ok=True)
    torch.save({
                'epoch': epoch+1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                }, 'outputs/last_model.pth')
def save_loss_plot(OUT_DIR, train_loss, val_loss):
    figure_1, train_ax = plt.subplots()
    figure_2, valid_ax = plt.subplots()
    train_ax.plot(train_loss, color='tab:blue')
    train_ax.set_xlabel('iterations')
    train_ax.set_ylabel('train loss')
    valid_ax.plot(val_loss, color='tab:red')
    valid_ax.set_xlabel('iterations')
    valid_ax.set_ylabel('validation loss')
    figure_1.savefig(f"{OUT_DIR}/train_loss.png")
    figure_2.savefig(f"{OUT_DIR}/valid_loss.png")
    print('SAVING PLOTS COMPLETE...')
    plt.close('all')

Prepare the dataset

In [ ]:
import torch
import cv2
import numpy as np
import os
import glob as glob
from xml.etree import ElementTree as et
from torch.utils.data import Dataset, DataLoader
# the dataset class
class CustomDataset(Dataset):
    def __init__(self, dir_path, width, height, classes, transforms=None):
        self.transforms = transforms
        self.dir_path = dir_path
        self.height = height
        self.width = width
        self.classes = classes

        # get all the image paths in sorted order
        self.image_paths = glob.glob(f"{self.dir_path}/*.jpg")
        self.all_images = [image_path.split(os.path.sep)[-1] for image_path in self.image_paths]
        self.all_images = sorted(self.all_images)

    def __getitem__(self, idx):
        # capture the image name and the full image path
        image_name = self.all_images[idx]
        image_path = os.path.join(self.dir_path, image_name)
        # read the image
        image = cv2.imread(image_path)
        # convert BGR to RGB color format
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image_resized = cv2.resize(image, (self.width, self.height))
        image_resized /= 255.0

        # capture the corresponding XML file for getting the annotations
        annot_filename = image_name[:-4] + '.xml'
        annot_file_path = os.path.join(self.dir_path, annot_filename)

        boxes = []
        labels = []
        tree = et.parse(annot_file_path)
        root = tree.getroot()

        # get the height and width of the image
        image_width = image.shape[1]
        image_height = image.shape[0]

        # box coordinates for xml files are extracted and corrected for image size given
        for member in root.findall('object'):
            # map the current object name to `classes` list to get...
            # ... the label index and append to `labels` list
            labels.append(self.classes.index(member.find('name').text))

            # xmin = left corner x-coordinates
            xmin = int(member.find('bndbox').find('xmin').text)
            # xmax = right corner x-coordinates
            xmax = int(member.find('bndbox').find('xmax').text)
            # ymin = left corner y-coordinates
            ymin = int(member.find('bndbox').find('ymin').text)
            # ymax = right corner y-coordinates
            ymax = int(member.find('bndbox').find('ymax').text)

            # resize the bounding boxes according to the desired `width`, `height`
            xmin_final = (xmin/image_width)*self.width
            xmax_final = (xmax/image_width)*self.width
            ymin_final = (ymin/image_height)*self.height
            yamx_final = (ymax/image_height)*self.height
            boxes.append([xmin_final, ymin_final, xmax_final, yamx_final])

        # bounding box to tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # area of the bounding boxes
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # no crowd instances
        iscrowd = torch.zeros((boxes.shape[0],), dtype=torch.int64)
        # labels to tensor
        labels = torch.as_tensor(labels, dtype=torch.int64)
        # prepare the final `target` dictionary
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["area"] = area
        target["iscrowd"] = iscrowd
        image_id = torch.tensor([idx])
        target["image_id"] = image_id
        target["name"]=image_name
        # apply the image transforms
        if self.transforms:
            sample = self.transforms(image = image_resized,
                                     bboxes = target['boxes'],
                                     labels = labels)
            image_resized = sample['image']
            target['boxes'] = torch.Tensor(sample['bboxes'])

        return image_resized, target

    def __len__(self):
        return len(self.all_images)

# prepare the final datasets and data loaders
def create_train_dataset():
    train_dataset = CustomDataset(TRAIN_DIR, RESIZE_TO, RESIZE_TO, CLASSES, get_train_transform())
    return train_dataset
def create_valid_dataset():
    valid_dataset = CustomDataset(VALID_DIR, RESIZE_TO, RESIZE_TO, CLASSES, get_valid_transform())
    return valid_dataset
def create_test_dataset():
    valid_dataset = CustomDataset(TEST_DIR, RESIZE_TO, RESIZE_TO, CLASSES, get_valid_transform())
    return valid_dataset
def create_train_loader(train_dataset, num_workers=0):
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=num_workers,
        collate_fn=collate_fn
    )
    return train_loader
def create_valid_loader(valid_dataset, num_workers=0, batch_size=BATCH_SIZE):
    valid_loader = DataLoader(
        valid_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=num_workers,
        collate_fn=collate_fn
    )
    return valid_loader

dataset = CustomDataset(
    TRAIN_DIR, RESIZE_TO, RESIZE_TO, CLASSES
)
print(f"Number of training images: {len(dataset)}")

# function to visualize a single sample
def visualize_sample(image, target):
    image=image*255
    for box_num in range(len(target['boxes'])):
        box = target['boxes'][box_num]
        label = CLASSES[target['labels'][box_num]]
        cv2.rectangle(
            image,
            (int(box[0]), int(box[1])), (int(box[2]), int(box[3])),
            (0, 255, 0), 2
        )
        cv2.putText(
            image, label, (int(box[0]), int(box[1]-5)),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2
        )
    image=image.astype(np.uint8)
    plt.imshow(image)
    plt.axis("off")
    plt.show()
    plt.figure()
NUM_SAMPLES_TO_VISUALIZE = 5
for i in range(NUM_SAMPLES_TO_VISUALIZE):
    image, target = dataset[i]
    visualize_sample(image, target)

Model setup. [Faster-rcnn](https://pytorch.org/vision/main/models/generated/torchvision.models.detection.fasterrcnn_resnet50_fpn.html?highlight=fasterrcnn_resnet50_fpn)

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
def create_model(num_classes):

    # load Faster RCNN pre-trained model
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

    # get the number of input features
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # define a new head for the detector with required number of classes
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

2. Train and validate


In [ ]:
import torch
import matplotlib.pyplot as plt
import time
from tqdm.auto import tqdm
plt.style.use('ggplot')

def train(train_data_loader, model):
    """
    This function trains the model on the provided training data loader.

    Args:
        train_data_loader (torch.utils.data.DataLoader): The data loader containing training images and targets.
        model (torch.nn.Module): The deep learning model to be trained.

    Returns:
        list: A list containing the training loss values for each iteration.
    """

    print('Training')

    global train_itr  # Assuming this tracks the current training iteration (potentially for logging)
    global train_loss_list  # Assuming this stores training loss values (potentially for monitoring)

    # Initialize tqdm progress bar for training loop visualization
    prog_bar = tqdm(train_data_loader, total=len(train_data_loader))

    for i, data in enumerate(prog_bar):
        # Clear optimizer gradients for each iteration
        optimizer.zero_grad()

        # Get images and targets from the current batch
        images, targets = data

        # Move images to the specified device (CPU or GPU)
        images = list(image.to(DEVICE) for image in images)

        # Move targets to the specified device and convert non-string elements to tensors on that device
        targets = [{k: v.to(DEVICE) for k, v in t.items() if not isinstance(v, str)} for t in targets]

        # Forward pass through the model to get loss dictionary
        loss_dict = model(images, targets)

        # Calculate the total loss by summing losses from the loss dictionary
        losses = sum(loss for loss in loss_dict.values())

        # Extract the loss value as a scalar
        loss_value = losses.item()

        # Append the loss value to the training loss list
        train_loss_list.append(loss_value)

        # Send the loss value to a visualization tool (e.g., TensorBoard) if applicable
        train_loss_hist.send(loss_value)  # Assuming train_loss_hist is a visualization tool

        # Perform backward pass to calculate gradients
        losses.backward()

        # Update model weights using optimizer step
        optimizer.step()

        # Update training iteration counter (potentially for logging)
        train_itr += 1

        # Update progress bar with current loss value
        prog_bar.set_description(desc=f"Loss: {loss_value:.4f}")

    return train_loss_list


def validate(valid_data_loader, model):
    """
    This function validates the model's performance on the provided validation data loader.

    Args:
        valid_data_loader (torch.utils.data.DataLoader): The data loader containing validation images and targets.
        model (torch.nn.Module): The deep learning model to be validated.

    Returns:
        list: A list containing the validation loss values for each iteration.
    """

    print('Validating')

    global val_itr  # Assuming this tracks the current validation iteration (potentially for logging)
    global val_loss_list  # Assuming this stores validation loss values (potentially for monitoring)

    # Initialize tqdm progress bar for validation loop visualization
    prog_bar = tqdm(valid_data_loader, total=len(valid_data_loader))

    for i, data in enumerate(prog_bar):
        # Get images and targets from the current batch
        images, targets = data

        # Move images to the specified device (CPU or GPU)
        images = list(image.to(DEVICE) for image in images)

        # Move targets to the specified device and convert non-string elements to tensors on that device
        targets = [{k: v.to(DEVICE) for k, v in t.items() if not isinstance(v, str)} for t in targets]

        # Disable gradient calculation for validation (saves memory and computation)
        with torch.no_grad():
            # Forward pass through the model to get loss dictionary
            loss_dict = model(images, targets)

            # Calculate the total loss by summing losses from the loss dictionary
            losses = sum(loss for loss in loss_dict.values())

            # Extract the loss value as a scalar
            loss_value = losses.item()

            # Append the loss value to the validation loss list
            val_loss_list.append(loss_value)

            # Send the loss value to a visualization tool (e.g., TensorBoard) if applicable
            val_loss_hist.send(loss_value)  # Assuming val_loss_hist is a visualization tool

            # Update validation iteration counter (potentially for logging)
            val_itr += 1

        # Update progress bar with current loss value
        prog_bar.set_description(desc=f"Loss: {loss_value:.4f}")

    return val_loss_list

Training code

In [ ]:
if __name__ == '__main__':
    train_dataset = create_train_dataset()
    valid_dataset = create_valid_dataset()
    test_dataset = create_test_dataset()
    train_loader = create_train_loader(train_dataset, NUM_WORKERS)
    valid_loader = create_valid_loader(valid_dataset, NUM_WORKERS)
    test_loader = create_valid_loader(test_dataset, NUM_WORKERS, batch_size=1)
    print(f"Number of training samples: {len(train_dataset)}")
    print(f"Number of validation samples: {len(valid_dataset)}\n")
    print(f"Number of test samples: {len(test_dataset)}\n")
    # initialize the model and move to the computation device
    model = create_model(num_classes=NUM_CLASSES)
    model = model.to(DEVICE)
    # get the model parameters
    params = [p for p in model.parameters() if p.requires_grad]
    # define the optimizer
    optimizer = torch.optim.Adam(params, lr=0.0001)
    # initialize the Averager class
    train_loss_hist = Averager()
    val_loss_hist = Averager()
    train_itr = 1
    val_itr = 1
    # train and validation loss lists to store loss values of all...
    # ... iterations till ena and plot graphs for all iterations
    train_loss_list = []
    val_loss_list = []
    # name to save the trained model with
    MODEL_NAME = 'model'
    # whether to show transformed images from data loader or not
    if VISUALIZE_TRANSFORMED_IMAGES:
        show_transformed_image(train_loader)
    # initialize SaveBestModel class
    save_best_model = SaveBestModel()
    # start the training epochs
    for epoch in range(NUM_EPOCHS):
        print(f"\nEPOCH {epoch+1} of {NUM_EPOCHS}")
        # reset the training and validation loss histories for the current epoch
        train_loss_hist.reset()
        val_loss_hist.reset()
        # start timer and carry out training and validation
        start = time.time()
        train_loss = train(train_loader, model)
        val_loss = validate(valid_loader, model)
        print(f"Epoch #{epoch+1} train loss: {train_loss_hist.value:.3f}")
        print(f"Epoch #{epoch+1} validation loss: {val_loss_hist.value:.3f}")
        end = time.time()
        print(f"Took {((end - start) / 60):.3f} minutes for epoch {epoch}")
        # save the best model till now if we have the least loss in the      # ... current epoch
        save_best_model(
            val_loss_hist.value, epoch, model, optimizer
        )
        # save the current epoch model
        save_model(epoch, model, optimizer)
        # save loss plot
        save_loss_plot(OUT_DIR, train_loss, val_loss)

        # sleep for 5 seconds after each epoch
        time.sleep(5)

Test the network

In [ ]:
import numpy as np
import cv2
import torch
import glob as glob
import os
import time
from copy import copy

# load the best model and trained weights
model = create_model(num_classes=NUM_CLASSES)
checkpoint = torch.load('outputs/best_model.pth', map_location=DEVICE)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE).eval()
# define the detection threshold...
# ... any detection having score below this will be discarded
detection_threshold = 0.8
# to count the total number of images iterated through
frame_count = 0
# to keep adding the FPS for each image
total_fps = 0
os.makedirs("inference_outputs/images",exist_ok=True)
# initialize tqdm progress bar
prog_bar = tqdm(test_loader, total=len(test_loader))

for i, data in enumerate(prog_bar):
    image, target = data
    image=image[0]
    target=target[0]
    orig_image = np.array(image).transpose(1,2,0)
    image=image.to(DEVICE)

    # get the image file name for saving output later on
    image_name = target["name"]
    bbox=target["boxes"]
    # print(image_name,target)

    # add batch dimension
    image = torch.unsqueeze(image, 0)
    start_time = time.time()
    with torch.no_grad():
        outputs = model(image.to(DEVICE))
    end_time = time.time()
    # get the current fps
    fps = 1 / (end_time - start_time)
    # add `fps` to `total_fps`
    total_fps += fps
    # increment frame count
    frame_count += 1
    # load all detection to CPU for further operations
    outputs = [{k: v.to('cpu') for k, v in t.items()} for t in outputs]
    # carry further only if there are detected boxes
    # Check if there are any detected bounding boxes
    if len(outputs[0]['boxes']) != 0:
        # Get boxes and scores from the output dictionary
        boxes = outputs[0]['boxes'].data.numpy()
        scores = outputs[0]['scores'].data.numpy()

        # Filter boxes based on a detection threshold (confidence score)
        boxes = boxes[scores >= detection_threshold].astype(np.int32)
        draw_boxes = boxes.copy()  # Copy boxes for drawing

        # Get predicted class names based on class indices
        pred_classes = [CLASSES[i] for i in outputs[0]['labels'].cpu().numpy()]

        # Loop through detected bounding boxes and draw them on the image
        for j, box in enumerate(draw_boxes):
            class_name = pred_classes[j]

            # Draw a green rectangle around the bounding box
            cv2.rectangle(orig_image, (int(box[0]), int(box[1])),
                          (int(box[2]), int(box[3])), (0, 255, 0), 2)

            # Write the predicted class name above the bounding box
            cv2.putText(orig_image, class_name, (int(box[0]), int(box[1] - 5)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # Display the image with bounding boxes and class names using matplotlib
        plt.imshow(orig_image)
        plt.axis("off")

        # Save the image with bounding boxes to a specified directory
        plt.savefig(f"inference_outputs/images/{image_name}.jpg", bbox_inches="tight",pad_inches=0)
        plt.show()
        # cv2.imwrite(f"inference_outputs/images/{image_name}.jpg", np.uint8(orig_image[:,:,::-1]*255))
        # Create a new figure for potentially further plotting needs
        plt.figure()
    print(f"Image {i+1} done...")
    print('-'*50)  # Print a separator for clarity

print('TEST PREDICTIONS COMPLETE')

# Close any open OpenCV windows
cv2.destroyAllWindows()

# Calculate and print the average FPS
avg_fps = total_fps / frame_count
print(f"Average FPS: {avg_fps:.3f}")


**ASSIGNMENT**

Display some images in the test set with the highest and lowest Intersection over Union (IoU) scores. Additionally, attempt to provide insights into why the model may encounter difficulties in detecting certain images.